In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import numpy as np
import configparser
import json
import requests
import xmltodict
from bs4 import BeautifulSoup
import time
import pickle
import json
import os
import gensim
import csv
import seaborn as sns
import smart_open
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import warnings
warnings.filterwarnings('ignore')

# retrieve episode descriptions

In [3]:
fname = 'feed.xml'
url = 'http://dataskeptic.com/feed.rss'

if not(os.path.isfile(fname)):
    print('fetching')
    r = requests.get(url)
    f = open(fname, 'wb')
    f.write(r.text.encode('utf-8'))
    f.close()

with open(fname) as fd:
    xml = xmltodict.parse(fd.read())

In [4]:
test = xml['rss']['channel']['item'][72]['description']
a = BeautifulSoup(test,'lxml').text
a

"I'm joined by Wes McKinney (@wesmckinn) and Hadley Wickham (@hadleywickham) on this episode to discuss their joint project Feather. Feather is a file format for storing data frames along with some metadata, to help with interoperability between languages. At the time of recording, libraries are available for R and Python, making it easy for data scientists working in these languages to quickly and effectively share datasets and collaborate."

In [5]:
episodes = xml['rss']['channel']['item']
descriptions = []
descToTitle = {}
descToLink = {}
descToNum = {}
l = len(episodes)
for episode in episodes:
    enclosure = episode['enclosure']
    
    desc = episode['description']
    desc = desc.replace(u'\xa0', u' ')
    desc = desc.replace(u'\n', u' ')
    desc = desc.replace(u'\xc2', u' ')

    
    desc = BeautifulSoup(desc, "lxml").text
    descriptions.append(desc)
    
    descToTitle[desc] = episode['title']
    descToLink[desc] = episode['link']
    descToNum[desc] = l
    l = l - 1

In [6]:
episodes[0].keys()

odict_keys(['title', 'pubDate', 'guid', 'link', 'itunes:image', 'description', 'content:encoded', 'enclosure', 'itunes:duration', 'itunes:explicit', 'itunes:keywords', 'itunes:subtitle', 'itunes:episodeType'])

In [7]:
result = {}
for desc in descriptions:
    info = {}
    info["link"] = descToLink[desc]
    info["title"] = descToTitle[desc]
    info["num"] = descToNum[desc]
    result[desc] = info
    

In [8]:
with open('./text/episodes_json.txt', 'w') as outfile:  
    json.dump(result, outfile)

In [9]:
for desc in descriptions:
    print(descToNum[desc])
    print(descToTitle[desc])
    print(descToLink[desc])    

179
Generative AI for Content Creation
https://dataskeptic.com/blog/episodes/2017/generative-ai-for-content-creation
178
[MINI] One Shot Learning
https://dataskeptic.com/blog/episodes/2017/one-shot-learning
177
Recommender Systems Live from FARCON 2017
https://dataskeptic.com/blog/episodes/2017/recommender-systems-live-from-farcon
176
[MINI] Long Short Term Memory
https://dataskeptic.com/blog/episodes/2017/long-short-term-memory
175
Zillow Zestimate
https://dataskeptic.com/blog/episodes/2017/zillow-zestimate
174
Cardiologist Level Arrhythmia Detection with CNNs
https://dataskeptic.com/blog/episodes/2017/cardiologist-level-arrhythmia-detection-with-cnns
173
[MINI] Recurrent Neural Networks
https://dataskeptic.com/blog/episodes/2017/recurrent-neural-networks
172
Project Common Voice
https://dataskeptic.com/blog/episodes/2017/project-common-voice
171
[MINI] Bayesian Belief Networks
http://dataskeptic.com/blog/episodes/2017/bayesian-belief-networks
170
pix2code
https://dataskeptic.com/blog

# Save description in txt file.

In [13]:
thefile = open('./text/episode_descs_titles.txt', 'w')

# for i, desc in enumerate(descriptions):
#     desc = desc.encode('utf-8').strip()
    
#     desc = "*"+ str(i)+str(desc).replace('\n', "") 
#     thefile.write("%s\n" % desc)

for i in range(len(descriptions)):
    desc = descriptions[i]
    title = descToTitle[desc]
    
    desc = desc.encode('utf-8').strip()
    desc = str(desc).replace('\n', "") 
    #print(desc)
    title = title.replace('[MINI]', "")
    title = title.encode('utf-8').strip()
    title = "*"+ str(i)+str(title).replace('\n', "") 
    #print(title)
    thefile.write("%s\n" % str(title+", "+desc))
    #print(title+" "+desc)
    

In [14]:
with open('./text/episode_descs_titles.txt', 'r') as f:
    i=0
    for line in f:
        i+=1
        print(i)
        print(line[0:40])

1
*0b'Generative AI for Content Creation',
2
*1b'One Shot Learning', b'One Shot Learn
3
*2b'Recommender Systems Live from FARCON
4
*3b'Long Short Term Memory', b'Thanks to
5
*4b'Zillow Zestimate', b'Zillow is a lea
6
*5b'Cardiologist Level Arrhythmia Detect
7
*6b'Recurrent Neural Networks', b'RNNs a
8
*7b'Project Common Voice', b"Thanks to o
9
*8b'Bayesian Belief Networks', b"A Bayes
10
*9b'pix2code', b'In this episode, Tony B
11
*10b'Conditional Independence', b"In sta
12
*11b'Estimating Sheep Pain with Facial R
13
*12b'CosmosDB', b'This episode collects 
14
*13b'The Vanishing Gradient', b'This epi
15
*14b'Doctor AI', b'hen faced with medica
16
*15b'Activation Functions', b'In a neura
17
*16b'MS Build 2017', b'This episode reca
18
*17b'Max-pooling', b"Max-pooling is a pr
19
*18b'Unsupervised Depth Perception', b'T
20
*19b'Convolutional Neural Networks', b"C
21
*20b'Multi-Agent Diverse Generative Adve
22
*21b'Generative Adversarial Networks', b
23
*22b'Opinion Polls for Presidential El

In [15]:
i # it should be 179 before 10/3/2017.

179

**Stop here and the rest can be done by rcm.py.**

# Use the word vectors trained from SO to represent episode descriptions.

## get word vectors trained from SO

In [16]:
key = 'word2vector_model_question_answer_200_6_2'
fname = './word_vec/'+key+".csv"
word_vecs_df = pd.read_csv(fname,index_col=0)
word_vecs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
a_,-0.502157,-1.231525,1.501073,-0.824721,0.591714,-0.488768,0.903779,-0.038645,0.459253,-0.485186,...,0.276586,-0.661979,0.297288,0.136056,-0.305163,1.396190,-0.239185,1.228788,0.333640,0.492632
a__,-0.044594,-0.024119,0.018766,0.024026,-0.028895,-0.022439,0.013539,0.007449,0.006484,-0.009192,...,0.024952,-0.038260,-0.004415,-0.019359,-0.053633,-0.015187,0.011295,-0.014313,0.038858,-0.030853
a_a,0.013455,-0.012830,-0.017656,0.014753,0.062009,-0.132435,0.068699,0.045951,0.074507,-0.035419,...,-0.028048,0.040826,0.069079,0.088605,0.082241,0.211491,0.126767,0.039316,-0.028075,-0.026345
a_adjusted,0.028670,0.020240,0.025201,-0.070229,0.012619,-0.078576,0.051204,0.040304,0.027123,0.021348,...,-0.016276,0.008707,-0.060311,0.086649,0.062956,0.068462,0.053034,0.037799,-0.016378,0.058798
a_and_b,0.114285,0.039049,0.213730,-0.080442,-0.088148,-0.197033,0.049060,0.106649,-0.017760,-0.002527,...,-0.111197,0.030113,-0.044530,0.090094,0.085388,0.165226,0.098310,0.086020,-0.143789,-0.047890


In [17]:
vocab = word_vecs_df.index
len(vocab)

87750

In [18]:
fname = './vocab_dict/vocab_dict_question_answer_200_6_2.csv'
with open(fname, 'r') as csv_file:
    reader = csv.reader(csv_file)
    vocab_dic = dict(reader)

In [19]:
for k, value in vocab_dic.items():
    vocab_dic[k] = int(value)
    

In [20]:
vocab_dic['a_a']

2

# Preprocessing the text in episode descriptions

In [21]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                
                yield gensim.utils.simple_preprocess(line)
                #This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line), [i])


In [22]:
fname = './text/episode_descs_titles.txt'
episode_desc_title_corpus = list(read_corpus(fname, tokens_only= True))

In [23]:
corpus = []
for desc in episode_desc_title_corpus:
    corpus.append(" ".join(desc))

In [24]:
len(corpus) 

179

In [25]:
corpus[0]

'generative ai for content creation last year the film development and production company end cue produced short film called sunspring that was entirely written by an artificial intelligence using neural networks more specifically it was authored by recurrent neural network rnn called long short term memory lstm according to end cue xe chief technical officer deb ray the company has come long way in improving the generative ai aspect of the bot in this episode deb ray joins host kyle polich to discuss how generative ai models are being applied in creative processes such as screenwriting their discussion also explores how data science for analyzing development projects such as financing and selecting scripts as well as optimizing the content production process'

In [26]:
fname = './text/episode_corpus.txt'
with open(fname, 'w') as f:
    for c in corpus:
        f.write("%s\n" % c)
  

# Get tf_idf features of episode descriptions

In [27]:
vectorizer = TfidfVectorizer(min_df=1,vocabulary = vocab_dic)
X = vectorizer.fit_transform(corpus)


Note: [vocabulary] Mapping or iterable, optional
Either a Mapping (e.g., a dict) where keys are terms and values are indices in the feature matrix, or an iterable over terms. If not given, a vocabulary is determined from the input documents.

Which words are in episode descriptions but not in the vocab of SO?


In [28]:
# What words are not in SO?
for i in range(179):
#     print(len(X[i,:].nonzero()[1]))
#     print(len(set(corpus[i].split(' '))))
    diff = set(corpus[i].split(' ')).difference(set(vocab))
    if len(diff) >5:
        print('*******************************************************')
        print(i)
        print(diff)  
        print(descriptions[i])

*******************************************************
23
{'iamzareenf', 'joytafty', 'dashboarding', 'aglpjrmp', 'periscopedata', 'rosevere', 'blueplastic', 'zareen', 'openhouse', 'zehr', 'jawbone', 'cataloging'}
No reliable, complete database cataloging home sales data at a transaction level is available for the average person to access. To a data scientist interesting in studying this data, our hands are complete tied. Opportunities like testing sociological theories, exploring economic impacts, study market forces, or simply research the value of an investment when buying a home are all blocked by the lack of easy access to this dataset. OpenHouse seeks to correct that by centralizing and standardizing all publicly available home sales transactional data. In this episode, we discuss the achievements of OpenHouse to date, and what plans exist for the future.     Check out the OpenHouse gallery.    I also encourage everyone to check out the project Zareen mentioned which was her Harr

*******************************************************
148
{'matchmaking', 'caretlibrary', 'withthomas', 'levi', 'pof', 'tslevi'}
Can algorithms help you find love? Many happy couples successfully brought together via online dating websites show us that data science can help you find love. I'm joined this week by Thomas Levi, Senior Data Scientist at Plenty of Fish, to discuss some of his work which helps people find one another as efficiently as possible. Matchmaking is a truly non-trivial problem, and one that's dynamically changing all the time as new users join and leave the "pool of fish". This episode explores the aspects of what makes this a tough problem and some of the ways POF has been successfully using data science to solve it, and continues to try to innovate with new techniques like interest matching. For his benevolent references, Thomas suggests readers check out All of Statistics as well as the caretlibrary for R. And for a self serving recommendation, follow him on t

Those words are either people's names, website names and they don't affect the main ideas and I feel it is ok to filter them. 

In [28]:
X.shape

(179, 87750)

# Get weighted doc vectors for all episode description

In [29]:
i=0
episode_desc_title_corpus[i]

['generative',
 'ai',
 'for',
 'content',
 'creation',
 'last',
 'year',
 'the',
 'film',
 'development',
 'and',
 'production',
 'company',
 'end',
 'cue',
 'produced',
 'short',
 'film',
 'called',
 'sunspring',
 'that',
 'was',
 'entirely',
 'written',
 'by',
 'an',
 'artificial',
 'intelligence',
 'using',
 'neural',
 'networks',
 'more',
 'specifically',
 'it',
 'was',
 'authored',
 'by',
 'recurrent',
 'neural',
 'network',
 'rnn',
 'called',
 'long',
 'short',
 'term',
 'memory',
 'lstm',
 'according',
 'to',
 'end',
 'cue',
 'xe',
 'chief',
 'technical',
 'officer',
 'deb',
 'ray',
 'the',
 'company',
 'has',
 'come',
 'long',
 'way',
 'in',
 'improving',
 'the',
 'generative',
 'ai',
 'aspect',
 'of',
 'the',
 'bot',
 'in',
 'this',
 'episode',
 'deb',
 'ray',
 'joins',
 'host',
 'kyle',
 'polich',
 'to',
 'discuss',
 'how',
 'generative',
 'ai',
 'models',
 'are',
 'being',
 'applied',
 'in',
 'creative',
 'processes',
 'such',
 'as',
 'screenwriting',
 'their',
 'discussion'

### How to get the weighted vectors of the episode descriptions?

....


<img src="pictures/tf_idf_matrix.png">
<img src="pictures/word_vec_df.png">


- For example, doc has three words: doc = [word1, word2, word3].
- vec_word_i = [d1, d2, ..., dn] 
- n = size in hidden layer.

- tf_idf_ji = tf_idf of word i in doc_j; Scale them such that sum_i tf_idf_ji = 1. 

- Then the vector of doc_j = sum_i (vec_word_i * tf_dif_ji) which is a vector with the same len as vec_word_i.

In [30]:
def get_doc_weighted_vec(i, doc_corpus , tf_idf = X, weighted = True): # ith documents. doc_corpus a list of words
    
    df = word_vecs_df 
    related_rows = df.loc[sorted(list(set(doc_corpus).intersection(set(vocab)))), :] 
    
    if weighted:
        weights = []
        ind = sorted(tf_idf[i,:].nonzero()[1])
        if sum([vectorizer.vocabulary_[related_rows.index[j]] != ind[j] for j in range(len(ind))]) != 0:
            print("words position don't match")
            return 
        for j in ind:
            weights.append(tf_idf[i,j])
        weights = np.array(weights)/sum(weights)
    else:
        weights = [1/related_rows.shape[0]] * related_rows.shape[0]
    
    if related_rows.shape[0] != len(weights):
        print(i)
        print(related_rows.shape[0])
        print(len(weights))
    
    result = related_rows.T * weights
    return result.sum(axis = 1)

In [31]:
episode_vec_weighted = []
total = len(descriptions)
for i in range(total):
    doc_corpus = episode_desc_title_corpus[i]
    episode_vec_weighted.append(get_doc_weighted_vec(i,doc_corpus))


### Save the episode weighted vectors

In [ ]:
def save_obj(obj, name ):
    with open('episode_vec/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name ):
    with open('episode_vec/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [ ]:
save_obj(episode_vec_weighted, "episode_vec_weighted_with_title")


In [33]:
len(episode_vec_weighted)

179

In [33]:
episode_vec_weighted_df = pd.DataFrame(episode_vec_weighted)
episode_vec_weighted_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.500611,-0.392125,0.031091,0.058523,0.132875,0.498406,0.123527,-0.011573,0.008225,-0.244095,...,-0.056908,-0.452368,-0.174699,-0.078086,0.015759,0.155416,-0.315453,0.723233,-0.093518,-0.089709
1,0.525857,0.068335,0.090763,0.311522,-0.015205,-0.002165,-0.055992,0.214757,-0.023833,-0.017694,...,-0.137174,-0.529182,-0.240740,-0.217705,0.237649,-0.538784,-0.306210,0.253044,0.062750,0.049728
2,0.523980,0.019395,0.003978,0.091436,0.262884,0.274046,0.499739,-0.304788,0.094896,-0.039237,...,-0.205848,-0.504070,-0.175757,-0.150721,-0.140069,0.166488,-0.278629,0.642518,0.405776,-0.229477
3,0.680434,-0.518742,0.615582,-0.139015,-0.009863,0.199846,0.092300,0.288420,0.227574,-0.332820,...,-0.411431,-0.327884,-0.859069,-0.106659,0.110672,-0.295466,-0.401121,0.459016,-0.208328,-0.124752
4,0.059128,-0.002912,0.022362,0.096436,-0.132251,0.503611,-0.005181,0.227001,-0.084239,-0.208569,...,0.007823,-0.148612,-0.557120,-0.281385,-0.041186,-0.312629,-0.352170,0.191202,-0.093452,-0.047086


In [34]:
episode_vec_weighted_df.shape

(179, 200)

In [35]:
episode_vec_weighted_df.to_csv("./episode_vec/episode_vec_weighted.csv")

In [36]:
test = pd.read_csv("./episode_vec/episode_vec_weighted.csv", index_col=0)
test.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.500611,-0.392125,0.031091,0.058523,0.132875,0.498406,0.123527,-0.011573,0.008225,-0.244095,...,-0.056908,-0.452368,-0.174699,-0.078086,0.015759,0.155416,-0.315453,0.723233,-0.093518,-0.089709
1,0.525857,0.068335,0.090763,0.311522,-0.015205,-0.002165,-0.055992,0.214757,-0.023833,-0.017694,...,-0.137174,-0.529182,-0.240740,-0.217705,0.237649,-0.538784,-0.306210,0.253044,0.062750,0.049728
2,0.523980,0.019395,0.003978,0.091436,0.262884,0.274046,0.499739,-0.304788,0.094896,-0.039237,...,-0.205848,-0.504070,-0.175757,-0.150721,-0.140069,0.166488,-0.278629,0.642518,0.405776,-0.229477
3,0.680434,-0.518742,0.615582,-0.139015,-0.009863,0.199846,0.092300,0.288420,0.227574,-0.332820,...,-0.411431,-0.327884,-0.859069,-0.106659,0.110672,-0.295466,-0.401121,0.459016,-0.208328,-0.124752
4,0.059128,-0.002912,0.022362,0.096436,-0.132251,0.503611,-0.005181,0.227001,-0.084239,-0.208569,...,0.007823,-0.148612,-0.557120,-0.281385,-0.041186,-0.312629,-0.352170,0.191202,-0.093452,-0.047086


In [38]:
Stop here.

SyntaxError: invalid syntax (<ipython-input-38-73bbe247de15>, line 1)

# Make recomendation: find related episode

## user's strings: 

some examples

In [43]:

# user_requests = [
#     "",
#     "Are there any episodes on Facial Recognition?",
#     "How to know whether the data I am using is valid for my purpose?",
#     "Could you recommend some episodes on decision tree and random forests?",
#     "Can you talk about Convolutional neural network and recurrent neural network?",
#     "Could you recommend some episodes on data science projects for beginners?",
#     "artificial intelligence",
#     "How can beginners in machine learning, who have finished their MOOCs in machine learning and deep learning, take it to the next level and get to the point of being able to read research papers & productively contribute in an industry?",
#     "What can artificial intelligence do for human beings? What is the future of artificial intelligence?",
#     "What is natural language processing? ",
#     "The error percentage of regression changes with change in the train and test data which I am deciding randomly. Cross validation can overcome this but how do I apply it for my regression model?",
#     "I have a precision recall curve for two separate algorithms. If I want to calculate the F-Measure I have to use the precision and recall values at a particular point on each curve. How is this point decided? For example on curve one there is a point where recall is 0.9 and precision is 0.87 and the other curve there is a point of recall at 0.95 and precision at 0.84. Alternatively, should I plot a F-measure curve for every precision recall value?",
#     "Suppose I want to make predictions of a response from predictors but I have some autocorrelation in the response variable. Under OLS this would be a problem as the residuals would have autocorrelation. What if I just want to predict the response and I use regularized least squares, like lasso or ridge or elastic net? I don't care about variances of the coefficients or anything of that nature as I'm not testing any hypotheses but I feel like I might be missing something.",
#     "Evaluating the quality of data.",
#     "I am interested in knowing musical stuff.",
#     "Is there any podcast on musical data and musical projects?",
#     "What is the trend of big data? What is big data? How to learn big data?",
#     "How to learn machine learning? What books or website do you recommend?",
#     "Looking for projects on criminal analysis? ",
#     "How to take advantage of Internet, computer,  cloud and other  platform in an effective way?",
#     "What are the most important knowledge in statistics or probability when doing machine learning?"
# ]  


user_requests = ["Are there any episodes on Facial Recognition?"]   

## Cosine Similarity

In [46]:
all_episode = episode_vec_weighted_df.values
with open('some examples with titles.txt', 'w') as f:
    for j in range(len(user_requests)):
        f.write("*****************************************************" + "\n")
        user_request = user_requests[j]
        user_request_corpus = gensim.utils.simple_preprocess(user_request)
        X_user = vectorizer.fit_transform([" ".join(user_request_corpus)])
        f.write(str(X_user.shape) + "\n")
        f.write('tf_idf is ' + str(X_user))
        user_weighted_vec = get_doc_weighted_vec(0, user_request_corpus, tf_idf = X_user, weighted = True)
        f.write("user_weighted_vec is \n" + str(user_weighted_vec[0:10]))
        cos_similarities = cosine_similarity(X=user_weighted_vec, Y=all_episode)

        cos_similarities = cos_similarities[0]
        cos_similarities.shape


        most_similar = cos_similarities.argsort()[-4:][::-1]
        f.write(str(most_similar) + "\n")

        threshold = 0.60
        f.write("User's request is: " + user_request + "\n" )
        for i in most_similar:

            if cos_similarities[i] > threshold:
                f.write("--------------------------"+str(cos_similarities[i])+"-----------------------------------\n")
                f.write( "\n")
                f.write(str(descToTitle[descriptions[i]]) + "\n")
                f.write(str(descToLink[descriptions[i]]) + "\n")
                f.write(str(descriptions[i].encode('utf-8')) + "\n")



    

Find the result at some example.txt.  

Since the number of all episode is handlable, let's have a look at the similarity between all episodes. By this, I also want to know the levels of the cosine similarities. 

In [ ]:
A = cosine_similarity(X=all_episode)
A.shape

In [ ]:
im = plt.imshow(A[20:40,20:40])
plt.colorbar(im)
plt.show()

# very diversity. so it is good.

## New string:

In [ ]:
user_request = input('what topics are interesting to you? ')


In [ ]:
print("Hello.", user_request)

In [ ]:
# to-do: reorganize the code and write a function recommend_episode.

def recommend_episode(string):
    all_episode = episode_vec_weighted_df.values

    
    print("*****************************************************" + "\n")
    user_request = string
    user_request_corpus = gensim.utils.simple_preprocess(user_request)
    X_user = vectorizer.fit_transform([" ".join(user_request_corpus)])
    #print(str(X_user.shape) + "\n")
    user_weighted_vec = get_doc_weighted_vec(0, user_request_corpus , tf_idf = X_user, weighted = True)
    cos_similarities = cosine_similarity(X=user_weighted_vec, Y=all_episode)

    cos_similarities = cos_similarities[0]
    cos_similarities.shape


    most_similar = cos_similarities.argsort()[-4:][::-1]
    #print(str(most_similar) + "\n")

    threshold = 0.60
    print("User's request is: " + user_request + "\n" )

    for i in most_similar:

        if cos_similarities[i] > threshold:
            print("--------------------The episode has cosine similarity is "+str(cos_similarities[i])+" with user's request-------------------------\n")
            print( "\n")
            print(str(descToTitle[descriptions[i]]) + "\n")
            print(str(descToLink[descriptions[i]]) + "\n")
            print(str(descriptions[i].encode('utf-8')) + "\n")



    
    return episode

In [ ]:
recommend_episode(user_request)

In [ ]:
'adboost' in vocab # so no matter how many times the word 'adboost' is in the string, it won't find 